In [35]:
import wfdb
import os
from glob import glob

In [36]:
def fetch_records(record_name, directory):
    record_details = os.path.join(directory, record_name)
    record = wfdb.rdrecord(record_details, physical=True)
    attributes = wfdb.rdann(record_details, extension='atr')

    return record, attributes

In [37]:
directory = '../files/mitdb/1.0.0'
data = {}

records = [os.path.basename(file).split('.')[0] for file in glob(os.path.join(directory, '*.dat'))]

for record_detail in records:
    record, attributes = fetch_records(record_name=record_detail, directory=directory)
    data[record_detail] =  {
            "record" : record,
            "attribute": attributes
        }

print(data)

{'100': {'record': <wfdb.io.record.Record object at 0x000002599E1362A0>, 'attribute': <wfdb.io.annotation.Annotation object at 0x000002599EFECC80>}, '101': {'record': <wfdb.io.record.Record object at 0x000002599EFEE780>, 'attribute': <wfdb.io.annotation.Annotation object at 0x000002599EFED8E0>}, '102': {'record': <wfdb.io.record.Record object at 0x000002599EFED790>, 'attribute': <wfdb.io.annotation.Annotation object at 0x000002599EFECA10>}, '103': {'record': <wfdb.io.record.Record object at 0x000002599EFECB30>, 'attribute': <wfdb.io.annotation.Annotation object at 0x000002599ECB4BC0>}, '104': {'record': <wfdb.io.record.Record object at 0x000002599EFED280>, 'attribute': <wfdb.io.annotation.Annotation object at 0x000002599EC9C1D0>}, '105': {'record': <wfdb.io.record.Record object at 0x000002599EC9D6A0>, 'attribute': <wfdb.io.annotation.Annotation object at 0x000002599EC9D9A0>}, '106': {'record': <wfdb.io.record.Record object at 0x000002599E261B50>, 'attribute': <wfdb.io.annotation.Annota

In [38]:
print(records)

['100', '101', '102', '103', '104', '105', '106', '107', '108', '109', '111', '112', '113', '114', '115', '116', '117', '118', '119', '121', '122', '123', '124', '200', '201', '202', '203', '205', '207', '208', '209', '210', '212', '213', '214', '215', '217', '219', '220', '221', '222', '223', '228', '230', '231', '232', '233', '234']


In [39]:
for file in glob(os.path.join(directory, '*.dat')):
    print(file)

../files/mitdb/1.0.0\100.dat
../files/mitdb/1.0.0\101.dat
../files/mitdb/1.0.0\102.dat
../files/mitdb/1.0.0\103.dat
../files/mitdb/1.0.0\104.dat
../files/mitdb/1.0.0\105.dat
../files/mitdb/1.0.0\106.dat
../files/mitdb/1.0.0\107.dat
../files/mitdb/1.0.0\108.dat
../files/mitdb/1.0.0\109.dat
../files/mitdb/1.0.0\111.dat
../files/mitdb/1.0.0\112.dat
../files/mitdb/1.0.0\113.dat
../files/mitdb/1.0.0\114.dat
../files/mitdb/1.0.0\115.dat
../files/mitdb/1.0.0\116.dat
../files/mitdb/1.0.0\117.dat
../files/mitdb/1.0.0\118.dat
../files/mitdb/1.0.0\119.dat
../files/mitdb/1.0.0\121.dat
../files/mitdb/1.0.0\122.dat
../files/mitdb/1.0.0\123.dat
../files/mitdb/1.0.0\124.dat
../files/mitdb/1.0.0\200.dat
../files/mitdb/1.0.0\201.dat
../files/mitdb/1.0.0\202.dat
../files/mitdb/1.0.0\203.dat
../files/mitdb/1.0.0\205.dat
../files/mitdb/1.0.0\207.dat
../files/mitdb/1.0.0\208.dat
../files/mitdb/1.0.0\209.dat
../files/mitdb/1.0.0\210.dat
../files/mitdb/1.0.0\212.dat
../files/mitdb/1.0.0\213.dat
../files/mitdb

A record can have the following records:

> * p_signal: The physical (float) signal values if physical=True
> * d_signal: The digital (integer) signal values if physical=False
> * fs: Sampling frequency
> * sig_len: Signal length
> * n_sig: Number of signals
> * sig_name: List of signal names
> * units: List of signal units
> * Other metadata like base_date, base_time, etc.

attributes have the following records:

> * sample: An Nx1 integer vector containing the annotation locations in samples relative to the beginning of the record.
> * symbol: An Nx1 character vector describing the annotation types (e.g., 'N' for normal beat, 'V' for premature ventricular contraction, etc.).
> * subtype: An Nx1 integer vector describing annotation subtypes.
> * chan: An Nx1 integer vector indicating the channel associated with each annotation.
> * num: An Nx1 integer vector containing the annotation numbers.
> * aux_note: An Nx1 cell array of strings containing auxiliary information for each annotation.
> * fs: The sampling frequency of the record (if available).


In [44]:
data['100']['record'].p_signal

array([[-0.145, -0.065],
       [-0.145, -0.065],
       [-0.145, -0.065],
       ...,
       [-0.675, -0.365],
       [-0.765, -0.335],
       [-1.28 ,  0.   ]])